In [2]:
import sys

import torch
import os
from datetime import datetime
import time
import random
import cv2
import pandas as pd
import numpy as np
import albumentations as A
import matplotlib.pyplot as plt
from albumentations.pytorch.transforms import ToTensorV2
from sklearn.model_selection import StratifiedKFold
from torch.utils.data import Dataset,DataLoader
from torch.utils.data.sampler import SequentialSampler, RandomSampler
from glob import glob
import pandas as pd
import gc
from effdet import get_efficientdet_config, EfficientDet, DetBenchTrain, DetBenchPredict
from effdet.efficientdet import HeadNet
import warnings

warnings.filterwarnings("ignore")

DATA_ROOT_PATH = 'test_images'
SEED = 42

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
seed_everything(SEED)

In [1]:
class PredictGlobalConfig:
    restore_from = '../experiments/effdet5_ds_only_impact_master_22_12_2020/best-checkpoint-007epoch.bin'
    num_workers = 4
    batch_size = 3
    
    #folder = '../experiments/effdet5_ds_only_impact_master_22_12_2020_ft'
    soft_nms = True
    
predict_config = PredictGlobalConfig

In [3]:
def get_valid_transforms():
    return A.Compose(
        [
            A.Resize(height=512, width=512, p=1.0),
            ToTensorV2(p=1.0),
        ],
        p=1.0,
    )

In [4]:
class DatasetRetriever(Dataset):
    def __init__(self, image_ids, transforms=None):
        super().__init__()
        self.image_ids = image_ids
        self.transforms = transforms

    def __getitem__(self, index: int):
        image_id = self.image_ids[index]
        image = cv2.imread(f'../data/train_images_all/{image_id}', cv2.IMREAD_COLOR).copy().astype(np.float32)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB).astype(np.float32)
        image /= 255.0
        if self.transforms:
            sample = {'image': image}
            sample = self.transforms(**sample)
            image = sample['image']
        return image, image_id

    def __len__(self) -> int:
        return self.image_ids.shape[0]

In [18]:
def make_predictions(images, score_threshold=0.5, all_boxes=False):
    images = torch.stack(images).to(DEVICE).float()
    box_list = []
    score_list = []
    with torch.no_grad():
        det = net(images)#, torch.tensor([1]*images.shape[0]).float().cuda())
        for i in range(images.shape[0]):
            boxes = det[i].detach().cpu().numpy()[:,:4]    
            scores = det[i].detach().cpu().numpy()[:,4]   
            label = det[i].detach().cpu().numpy()[:,5]
            # useing only label = 2
            if all_boxes:
                indexes = np.where((scores > score_threshold) )[0]
            else:
                indexes = np.where((scores > score_threshold) & (label == 2))[0]
            #boxes[:, 2] = boxes[:, 2] + boxes[:, 0]
            #boxes[:, 3] = boxes[:, 3] + boxes[:, 1]
            box_list.append(boxes[indexes])
            score_list.append(scores[indexes])
    return box_list, score_list


In [6]:
DEVICE = torch.device('cuda:1') 
SIZE = 512

In [8]:
def load_net(predict_config):
    config = get_efficientdet_config('tf_efficientdet_d5')
    config.image_size = [512, 512]
    config.norm_kwargs = dict(eps=0.001, momentum=0.01)
    config.soft_nms = predict_config.soft_nms
    net = EfficientDet(config, pretrained_backbone=False)
    checkpoint = torch.load(predict_config.restore_from, map_location=DEVICE)
    
    net.class_net = HeadNet(config, num_outputs=config.num_classes)
    
    net.reset_head(num_classes=2)
    net.load_state_dict(checkpoint['model_state_dict'])
    
    net = DetBenchPredict(net)
    net.eval();
    return net.to(DEVICE)
#if IS_PRIVATE:
net = load_net(predict_config)

In [9]:
val_df = pd.read_csv('../data/evaluation_labels.csv')
imgs = val_df.image_name.unique()

In [10]:
val_df.loc[(val_df.impact == 2) & (val_df.confidence <= 1), 'impact'] = 1

In [11]:
val_df.loc[(val_df.impact == 2) & (val_df.visibility <= 0), 'impact'] = 1

In [12]:
dataset = DatasetRetriever(
    image_ids=np.array(imgs),
    transforms=get_valid_transforms()
)

def collate_fn(batch):
    return tuple(zip(*batch))

data_loader = DataLoader(
    dataset,
    batch_size=16,
    shuffle=False,
    num_workers=4,
    drop_last=False,
    collate_fn=collate_fn
)

In [13]:
from timm.models.efficientnet import tf_efficientnet_b4_ns, tf_efficientnet_b3_ns, \
    tf_efficientnet_b5_ns, tf_efficientnet_b2_ns, tf_efficientnet_b6_ns, tf_efficientnet_b7_ns
from nfl_impact_detection.scripts.classifier.model import ImpactClassifier
class PredictionClassifierConfig:
    restore_from = '../experiments/classifiers/effnet7_baseline_aug_3/best-checkpoint-005epoch.bin'
    num_workers = 4
    margin = 2.5
    encoder = {
                "name" : tf_efficientnet_b7_ns,
                "features": 2560
              }
    
    

In [14]:
def get_valid_classifier_transforms():
    return A.Compose(
        [   
            A.Resize(height=128, width=128, p=1),
            A.Normalize(),
            ToTensorV2(p=1.0),
        ]
    )

In [15]:
classifier_config = PredictionClassifierConfig

In [16]:
classifier = ImpactClassifier(encoder=classifier_config.encoder)

checkpoint = torch.load(classifier_config.restore_from,  map_location=DEVICE)
classifier.load_state_dict(checkpoint['model_state_dict'])
classifier.eval();

In [20]:

def get_crop_size_by_box(box):
    x, y, w, h = box
    crop_size = max(w, h) // 2
    crop_size += crop_size * classifier_config.margin
    crop_size = round(crop_size)
    return crop_size

def get_cropped_samples(image_idx, boxes, scores, transforms=None):
    image = cv2.imread(f'../data/train_images_all/{image_idx}', cv2.IMREAD_COLOR).copy().astype(np.float32)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB).astype(np.float32)
    
    crop_sizes = [get_crop_size_by_box(box) for box in boxes]
    max_crop_size = max(crop_sizes)

    im_size = image.shape
    pad_image = np.zeros((im_size[0]+2*max_crop_size, im_size[1]+2*max_crop_size, im_size[2]))
    pad_image[max_crop_size:pad_image.shape[0]-max_crop_size, 
            max_crop_size:pad_image.shape[1]-max_crop_size] = image

    outputs = []
    for i, box in enumerate(boxes):
        crop_size = crop_sizes[i]
        x, y, w, h = box
        center = [0,0]
        center[0] = x + w // 2 + max_crop_size
        center[1] = y + h // 2 + max_crop_size

        crop_img = pad_image[center[1]-crop_size:center[1]+crop_size,
                        center[0]-crop_size:center[0]+crop_size].astype('uint8')
        if transforms:
            sample = transforms(**{
                'image': crop_img
            })     
        sample['box'] = box
        sample['detector_score'] = scores[i]
        outputs.append(sample)
    return outputs

In [21]:
import tqdm

all_boxes=False
result_image_ids = []
results_boxes = []
results_scores = []
results_detector_scores = []
results_classifier_scores = []
for images, image_ids in tqdm.tqdm_notebook(data_loader):
    box_list, score_list = make_predictions(images, score_threshold=0.3, all_boxes=all_boxes)
    #predictions = make_tta_predictions(images)
    for i, image in enumerate(images):
        #boxes, scores, labels = run_wbf(predictions, image_index=i)

        #score_threshold = STAGE1_SUPPRESS
        #indexes = np.where(scores>score_threshold)
        #boxes = boxes[indexes]
        #scores = scores[indexes]
        
        boxes = box_list[i]
        scores = score_list[i]
        detector_scores = np.array([])
        classifier_scores = np.array([])
        image_id = image_ids[i]
    
        boxes[:, 0] = (boxes[:, 0] * 1280 / 512)
        boxes[:, 1] = (boxes[:, 1] * 720 / 512)
        boxes[:, 2] = (boxes[:, 2] * 1280 / 512)
        boxes[:, 3] = (boxes[:, 3] * 720 / 512)
        boxes[:, 2] = boxes[:, 2] - boxes[:, 0]
        boxes[:, 3] = boxes[:, 3] - boxes[:, 1]
        boxes = boxes.astype(np.int32)
        boxes[:, 0] = boxes[:, 0].clip(min=0, max=1280-1)
        boxes[:, 2] = boxes[:, 2].clip(min=0, max=1280-1)
        boxes[:, 1] = boxes[:, 1].clip(min=0, max=720-1)
        boxes[:, 3] = boxes[:, 3].clip(min=0, max=720-1)
        
        if len(boxes)>0:    
            cropped_samples = get_cropped_samples(
                image_id,
                boxes,
                scores,
                transforms=get_valid_classifier_transforms())
            
            boxes=[]
            scores=[]
            classifier_scores=[]
            detector_scores=[]
            tta_samples = torch.stack(
                [sample['image'] for sample in cropped_samples] +
                [sample['image'].flip(0) for sample in cropped_samples] +
                [sample['image'].flip(2) for sample in cropped_samples] +
                [sample['image'].flip(0).flip(2) for sample in cropped_samples]
                )

            classifier_preds = torch.sigmoid(classifier(tta_samples)).detach().numpy() #flip(2) for horizontal
            #classifier_preds2 = torch.sigmoid(classifier(torch.stack([sample['image'].flip(2) for sample in cropped_samples]))).detach().numpy()
            for i, sample in enumerate(cropped_samples):
                #classifier_score = torch.sigmoid(classifier(sample['image'])).detach().numpy()
                l = len(cropped_samples)
                classifier_score = (2*classifier_preds[i]+classifier_preds[i+l]+classifier_preds[i+l*2]+classifier_preds[i+l*3]) / 5.
                boxes.append(sample['box'])
                classifier_scores.append(classifier_score[0])
                detector_scores.append(sample['detector_score'])
                scores.append(classifier_score[0] * sample['detector_score'])

            boxes = np.array(boxes)
            scores = np.array(scores)
            detector_scores = np.array(detector_scores)
            classifier_scores = np.array(classifier_scores)

        results_boxes.append(boxes)
        results_scores.append(scores)
        results_detector_scores.append(detector_scores)
        results_classifier_scores.append(classifier_scores)
        result_image_ids += [image_id]*len(boxes)

KeyboardInterrupt: 

In [204]:
box_df = pd.DataFrame(np.concatenate(results_boxes), columns=['left', 'top', 'width', 'height'])
test_df = pd.DataFrame({'scores':np.concatenate(results_scores), 'image_name':result_image_ids})
test_df_detector = pd.DataFrame({'detector_scores':np.concatenate(results_detector_scores)})
test_df_classifier = pd.DataFrame({'classifier_scores':np.concatenate(results_classifier_scores)})

test_df = pd.concat([test_df, test_df_detector, test_df_classifier, box_df], axis=1)
test_df['frame'] = test_df.image_name.apply(lambda x: int(x.split('_')[-1].split('.')[0]))
test_df['video'] = test_df.image_name.apply(lambda x: '_'.join(x.split('_')[:3])+'.mp4' )
test_df.shape

(1910, 10)

In [205]:
test_df.head()

,scores,image_name,detector_scores,classifier_scores,left,top,width,height,frame,video
0,0.168355,57583_000082_Endzone_35.png,0.215000,0.783049,564,326,21,21,35,57583_000082_Endzone.mp4
1,0.183240,57583_000082_Endzone_36.png,0.213280,0.859153,564,326,21,20,36,57583_000082_Endzone.mp4
2,0.076194,57583_000082_Endzone_40.png,0.203800,0.373868,443,320,21,17,40,57583_000082_Endzone.mp4
3,0.167738,57583_000082_Endzone_42.png,0.221987,0.755618,442,319,20,16,42,57583_000082_Endzone.mp4
4,0.202959,57583_000082_Endzone_43.png,0.276977,0.732766,443,320,19,15,43,57583_000082_Endzone.mp4


In [167]:
def nms(scores, frames, x1, y1, x2, y2, thresh=0.5, fr_th=200):
    areas = (x2 - x1 + 1) * (y2 - y1 + 1)
    order = scores.argsort()[::-1]

    keep = []
    while order.size > 0:
        i = order[0]
        keep.append(i)
        xx1 = np.maximum(x1[i], x1[order[1:]])
        yy1 = np.maximum(y1[i], y1[order[1:]])
        xx2 = np.minimum(x2[i], x2[order[1:]])
        yy2 = np.minimum(y2[i], y2[order[1:]])

        w = np.maximum(0.0, xx2 - xx1 + 1)
        h = np.maximum(0.0, yy2 - yy1 + 1)
        inter = w * h
        ovr = inter / (areas[i] + areas[order[1:]] - inter)
        frm = frames[order[1:]]

        inds = np.where((ovr <= thresh) | (abs(frm - frames[i]) > fr_th))[0]
        # inds = np.where(ovr <= thresh)[0]
        order = order[inds + 1]

    return keep

In [168]:
def filter_by_nms(test_df, tr=0.5, fr_tr=200):
    indices = []
    leave = 0
    for vid in test_df.video.unique():
        info = test_df[test_df.video ==vid]
        #print(info.columns)
        x1 = info['left'].values
        y1 = info['top'].values
        x2 = x1 + info['width'].values
        y2 = y1 + info['height'].values
        res = nms(info['scores'].values, info['frame'].values, x1, y1, x2, y2, tr, fr_tr)
        leave += len(res)
        #print(vid,' to_drop', info.shape[0]-len(res))
        indices_to_drop = info.index[list(set(list(range(info.shape[0]))) - set(res))]
        indices.extend(list(indices_to_drop))
    test_df= test_df.drop(index = indices)
    #print(leave, len(indices))
    return test_df
    
    

In [169]:
def iou(bbox1, bbox2):
    bbox1 = [float(x) for x in bbox1]
    bbox2 = [float(x) for x in bbox2]

    (x0_1, y0_1, x1_1, y1_1) = bbox1
    (x0_2, y0_2, x1_2, y1_2) = bbox2

    # get the overlap rectangle
    overlap_x0 = max(x0_1, x0_2)
    overlap_y0 = max(y0_1, y0_2)
    overlap_x1 = min(x1_1, x1_2)
    overlap_y1 = min(y1_1, y1_2)

    # check if there is an overlap
    if overlap_x1 - overlap_x0 <= 0 or overlap_y1 - overlap_y0 <= 0:
        return 0

    # if yes, calculate the ratio of the overlap to each ROI size and the unified size
    size_1 = (x1_1 - x0_1) * (y1_1 - y0_1)
    size_2 = (x1_2 - x0_2) * (y1_2 - y0_2)
    size_intersection = (overlap_x1 - overlap_x0) * (overlap_y1 - overlap_y0)
    size_union = size_1 + size_2 - size_intersection

    return size_intersection / size_union


def precision_calc(gt_boxes, pred_boxes):
    cost_matix = np.ones((len(gt_boxes), len(pred_boxes)))
    for i, box1 in enumerate(gt_boxes):
        for j, box2 in enumerate(pred_boxes):
            dist = abs(box1[0] - box2[0])
            if dist > 4:
                continue
            iou_score = iou(box1[1:], box2[1:])

            if iou_score < 0.35:
                continue
            else:
                cost_matix[i, j] = 0

    row_ind, col_ind = linear_sum_assignment(cost_matix)
    fn = len(gt_boxes) - row_ind.shape[0]
    fp = len(pred_boxes) - col_ind.shape[0]
    tp = 0
    for i, j in zip(row_ind, col_ind):
        if cost_matix[i, j] == 0:
            tp += 1
        else:
            fp += 1
            fn += 1
    return tp, fp, fn

In [170]:
from scipy.optimize import linear_sum_assignment
def get_boxes(df):
    arr = df[['frame', 'left', 'top', 'width', 'height']].to_numpy()
    arr[:, 3] = arr[:,1] + arr[:, 3]
    arr[:, 4] = arr[:,2] + arr[:, 4]
    return arr

In [171]:
def calculate_metrics(test_df, val_df):
    ftp, ffp, ffn = [], [], []
    for vid in val_df.video.unique():
        #if vid.split('_')[-1].split('.')[0] == 'Endzone':
        #if vid.split('_')[-1].split('.')[0] == 'Sideline':
        #    continue
        pred_boxes = get_boxes(test_df[test_df.video == vid])
        gt_boxes = get_boxes(val_df[(val_df.video == vid) & (val_df.impact == 2)])
        tp, fp, fn = precision_calc(gt_boxes, pred_boxes)
        precision = tp / (tp + fp + 1e-6)
        recall = tp / (tp + fn + 1e-6)
        f1_score = 2 * (precision * recall) / (precision + recall + 1e-6)
        print(f'{vid}:  TP: {tp}, FP: {fp}, FN: {fn}, PRECISION: {precision:.2f}, RECALL: {recall:.2f}, F1 SCORE: {f1_score:.2f}')
        ftp.append(tp)
        ffp.append(fp)
        ffn.append(fn)
    tp = np.sum(ftp)
    fp = np.sum(ffp)
    fn = np.sum(ffn)
    precision = tp / (tp + fp + 1e-6)
    recall = tp / (tp + fn + 1e-6)
    f1_score = 2 * (precision * recall) / (precision + recall + 1e-6)
    return tp, fp, fn, precision, recall, f1_score
    

In [172]:
# this code for checking
# check_video_df = test_df[test_df.video == '58095_004022_Endzone.mp4']
# check_video_df = val_df[(val_df.video == '58095_004022_Endzone.mp4') & (val_df.impact == 2)]
# check_video_df['frame'] = check_video_df.image_name.str.split('_').str[3].str.replace('.png','').astype(int)

In [173]:
# check_video_df = check_video_df[check_video_df.frame == 176]
# fig, ax = plt.subplots(1, 1, figsize=(16, 8))
# sample = cv2.cvtColor(cv2.imread('../data/train_images_all/58095_004022_Endzone_176.png'), cv2.COLOR_BGR2RGB)
# for index, row in check_video_df.iterrows():
#     print(index)
#     x1 = row['left']
#     y1 = row['top']
#     x2 = x1 + row['width']
#     y2 = y1 + row['height']
    
#     #cv2.rectangle(sample, (x1, y1), (x2, y2), (1, 0, 0), 3)
# ax.set_axis_off()
# ax.imshow(sample);

In [174]:
test_df

,scores,image_name,detector_scores,classifier_scores,left,top,width,height,frame,video
0,0.026172,57583_000082_Endzone_8.png,0.300132,0.087201,592,325,19,24,8,57583_000082_Endzone.mp4
1,0.028701,57583_000082_Endzone_9.png,0.304256,0.094332,591,325,20,24,9,57583_000082_Endzone.mp4
2,0.163126,57583_000082_Endzone_18.png,0.345572,0.472045,589,325,22,23,18,57583_000082_Endzone.mp4
3,0.205627,57583_000082_Endzone_32.png,0.343757,0.598176,564,328,23,21,32,57583_000082_Endzone.mp4
4,0.216342,57583_000082_Endzone_32.png,0.328258,0.659063,582,312,21,24,32,57583_000082_Endzone.mp4
...,...,...,...,...,...,...,...,...,...,...
5937,0.212036,58103_003494_Sideline_188.png,0.357808,0.592598,736,386,17,18,188,58103_003494_Sideline.mp4
5938,0.177606,58103_003494_Sideline_189.png,0.360596,0.492535,737,387,16,17,189,58103_003494_Sideline.mp4
5939,0.035858,58103_003494_Sideline_190.png,0.303808,0.118029,776,303,17,16,190,58103_003494_Sideline.mp4
5940,0.044243,58103_003494_Sideline_191.png,0.348191,0.127065,776,301,17,16,191,58103_003494_Sideline.mp4


In [206]:
NMS_THRESHOLD = 0.35
CLS_THRESHOLD = 0.89
DETECTOR_THRESHOLD = 0.38
NMS_PARAM = 5000

In [176]:
# CODE FOR GRID SEARCH
"""
max_f1_score = 0
for CLS_THRESHOLD in tqdm.tqdm(np.array((range(800,1000,10))) / 1000):
    for DETECTOR_THRESHOLD in np.array((range(350,550,10))) / 1000:
        for NMS_THRESHOLD in np.array((range(350,850,50))) / 1000:
            test_df_preditions = test_df[test_df.classifier_scores>CLS_THRESHOLD][test_df.detector_scores>DETECTOR_THRESHOLD]
            test_df_filtered_by_nms = filter_by_nms(test_df_preditions, NMS_THRESHOLD, 5000)
            tp, fp, fn, precision, recall, f1_score = calculate_metrics(test_df_filtered_by_nms, val_df)
            if f1_score>max_f1_score:
                max_f1_score=f1_score
                save_DETECTOR_THRESHOLD = DETECTOR_THRESHOLD
                save_CLS_THRESHOLD = CLS_THRESHOLD
                save_NMS_THRESHOLD = NMS_THRESHOLD
"""

'\nmax_f1_score = 0\nfor CLS_THRESHOLD in tqdm.tqdm(np.array((range(800,1000,10))) / 1000):\n    for DETECTOR_THRESHOLD in np.array((range(350,550,10))) / 1000:\n        for NMS_THRESHOLD in np.array((range(350,850,50))) / 1000:\n            test_df_preditions = test_df[test_df.classifier_scores>CLS_THRESHOLD][test_df.detector_scores>DETECTOR_THRESHOLD]\n            test_df_filtered_by_nms = filter_by_nms(test_df_preditions, NMS_THRESHOLD, 5000)\n            tp, fp, fn, precision, recall, f1_score = calculate_metrics(test_df_filtered_by_nms, val_df)\n            if f1_score>max_f1_score:\n                max_f1_score=f1_score\n                save_DETECTOR_THRESHOLD = DETECTOR_THRESHOLD\n                save_CLS_THRESHOLD = CLS_THRESHOLD\n                save_NMS_THRESHOLD = NMS_THRESHOLD\n'

In [208]:
test_df_preditions = test_df[test_df.classifier_scores>CLS_THRESHOLD][test_df.detector_scores>DETECTOR_THRESHOLD]
test_df_preditions

,scores,image_name,detector_scores,classifier_scores,left,top,width,height,frame,video
6,0.247238,57583_000082_Endzone_44.png,0.276593,0.893869,420,311,21,26,44,57583_000082_Endzone.mp4
7,0.287932,57583_000082_Endzone_45.png,0.311507,0.924318,423,312,20,24,45,57583_000082_Endzone.mp4
8,0.228267,57583_000082_Endzone_45.png,0.253990,0.898725,445,322,18,13,45,57583_000082_Endzone.mp4
9,0.303664,57583_000082_Endzone_46.png,0.329177,0.922495,426,313,19,22,46,57583_000082_Endzone.mp4
10,0.189574,57583_000082_Endzone_46.png,0.210522,0.900497,443,320,18,14,46,57583_000082_Endzone.mp4
...,...,...,...,...,...,...,...,...,...,...
1881,0.196533,58103_003494_Sideline_63.png,0.213840,0.919065,743,346,15,16,63,58103_003494_Sideline.mp4
1882,0.193788,58103_003494_Sideline_65.png,0.211032,0.918286,741,347,15,16,65,58103_003494_Sideline.mp4
1894,0.199695,58103_003494_Sideline_82.png,0.223266,0.894428,878,173,17,18,82,58103_003494_Sideline.mp4
1897,0.192379,58103_003494_Sideline_114.png,0.212519,0.905234,768,191,15,16,114,58103_003494_Sideline.mp4


In [178]:
#TTA horizontal and channel flip
calculate_metrics(test_df_preditions, val_df)

57583_000082_Endzone.mp4:  TP: 3, FP: 22, FN: 16, PRECISION: 0.12, RECALL: 0.16, F1 SCORE: 0.14
57583_000082_Sideline.mp4:  TP: 2, FP: 22, FN: 16, PRECISION: 0.08, RECALL: 0.11, F1 SCORE: 0.10
57586_004152_Endzone.mp4:  TP: 6, FP: 22, FN: 2, PRECISION: 0.21, RECALL: 0.75, F1 SCORE: 0.33
57586_004152_Sideline.mp4:  TP: 4, FP: 23, FN: 5, PRECISION: 0.15, RECALL: 0.44, F1 SCORE: 0.22
57679_003316_Endzone.mp4:  TP: 7, FP: 56, FN: 12, PRECISION: 0.11, RECALL: 0.37, F1 SCORE: 0.17
57679_003316_Sideline.mp4:  TP: 3, FP: 3, FN: 14, PRECISION: 0.50, RECALL: 0.18, F1 SCORE: 0.26
57680_002206_Endzone.mp4:  TP: 12, FP: 55, FN: 7, PRECISION: 0.18, RECALL: 0.63, F1 SCORE: 0.28
57680_002206_Sideline.mp4:  TP: 8, FP: 71, FN: 11, PRECISION: 0.10, RECALL: 0.42, F1 SCORE: 0.16
57906_000718_Endzone.mp4:  TP: 5, FP: 35, FN: 15, PRECISION: 0.12, RECALL: 0.25, F1 SCORE: 0.17
57906_000718_Sideline.mp4:  TP: 2, FP: 15, FN: 14, PRECISION: 0.12, RECALL: 0.12, F1 SCORE: 0.12
57911_000147_Endzone.mp4:  TP: 4, FP: 

(97, 705, 301, 0.12094763077188575, 0.24371859235246585, 0.16166622307065998)

In [210]:
test_df_filtered_by_nms = filter_by_nms(test_df_preditions, NMS_THRESHOLD, NMS_PARAM)

In [180]:
calculate_metrics(test_df_filtered_by_nms, val_df)

57583_000082_Endzone.mp4:  TP: 3, FP: 4, FN: 16, PRECISION: 0.43, RECALL: 0.16, F1 SCORE: 0.23
57583_000082_Sideline.mp4:  TP: 1, FP: 6, FN: 17, PRECISION: 0.14, RECALL: 0.06, F1 SCORE: 0.08
57586_004152_Endzone.mp4:  TP: 5, FP: 3, FN: 3, PRECISION: 0.62, RECALL: 0.62, F1 SCORE: 0.62
57586_004152_Sideline.mp4:  TP: 3, FP: 1, FN: 6, PRECISION: 0.75, RECALL: 0.33, F1 SCORE: 0.46
57679_003316_Endzone.mp4:  TP: 7, FP: 5, FN: 12, PRECISION: 0.58, RECALL: 0.37, F1 SCORE: 0.45
57679_003316_Sideline.mp4:  TP: 3, FP: 1, FN: 14, PRECISION: 0.75, RECALL: 0.18, F1 SCORE: 0.29
57680_002206_Endzone.mp4:  TP: 10, FP: 14, FN: 9, PRECISION: 0.42, RECALL: 0.53, F1 SCORE: 0.47
57680_002206_Sideline.mp4:  TP: 5, FP: 7, FN: 14, PRECISION: 0.42, RECALL: 0.26, F1 SCORE: 0.32
57906_000718_Endzone.mp4:  TP: 5, FP: 11, FN: 15, PRECISION: 0.31, RECALL: 0.25, F1 SCORE: 0.28
57906_000718_Sideline.mp4:  TP: 2, FP: 5, FN: 14, PRECISION: 0.29, RECALL: 0.12, F1 SCORE: 0.17
57911_000147_Endzone.mp4:  TP: 3, FP: 9, FN: 

(78, 144, 320, 0.3513513497686876, 0.19597989900507562, 0.25161244270635474)